In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
os.chdir("../..")


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time

from collections import namedtuple
from IPython.display import clear_output
from tqdm.notebook import tqdm
from numpy.typing import NDArray

from scipy.optimize import minimize
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit import Parameter, Instruction
from qiskit.circuit.library import RXGate, RYGate, RZGate, CXGate, CZGate, IGate
from qiskit_aer import AerSimulator


In [4]:
from qml.model.gate import GateInfo, get_gateset, Gate


In [44]:

class Unit:
    VALUE_MAX = 2 * np.pi

    def __init__(
            self,
            name: str,
            gates: list[Gate],
            params: list[Parameter],
            values: list[float] | NDArray,
    ):
        self._name = name
        self._gates = gates
        self._params = params
        self._values = np.asarray(values) % self.VALUE_MAX

    def feed_dict(self, values=None) -> dict[str, float]:
        if values is None:
            values = self.values
        if not hasattr(values, "__len__"):
            values = [values]
        assert len(values) == len(
            self.parameters), f"Length of values {len(values)} must be equal to number of parameters {len(self.values)}"

        feed_dict = dict()
        for param, value in zip(self._params, values):
            feed_dict |= {
                param.name: value
            }
        return feed_dict

    def apply_to_qc(self, qc: QuantumCircuit) -> QuantumCircuit:
        for gate in self._gates:
            gate.apply_to_qc(qc)
        return qc

    @classmethod
    def generate_random_unit(
            cls,
            name: str,
            num_qubit: int,
            num_gate: int,
            gateset: dict[str, GateInfo] = None,
    ):
        if gateset is None:
            gateset = get_gateset(num_qubit)

        # select gate at random
        gate_names_on_set = list(gateset.keys())
        gate_names = np.random.choice(gate_names_on_set, size=num_gate, replace=True)

        # select qubits to apply gates
        qubits = np.random.randint(0, num_qubit, size=num_gate)

        return cls.new_with_gate_names_and_qubits(name, gate_names, qubits, gateset)

    @classmethod
    def new_with_gate_names_and_qubits(
            cls,
            name: str,
            gate_names: list[str],
            qubits: list[int],
            gateset: dict[str, GateInfo]
    ):
        gate_infos = [gateset[gate_name] for gate_name in gate_names]

        # build instance of gates and parameters
        gates = []
        params = []
        for gate_info, qubit in zip(gate_infos, qubits):
            if not gate_info.trainable:
                gates.append(Gate.new_with_info(gate_info, qubit))
                continue

            pname = f"param_{len(params)}"
            if name is not None:
                pname = name + "_" + pname
            param = Parameter(pname)
            params.append(param)
            gates.append(Gate.new_with_info(gate_info, qubit, param))

        # initialize parameter values
        values = np.zeros_like(params)

        return cls(name, gates, params, values)

    @property
    def values(self):
        return self._values.copy()

    @values.setter
    def values(self, values):
        assert len(values) == len(
            self.values), f"Length of values {len(values)} must be equal to number of parameters {len(self.values)}"
        values = np.asarray(values)
        values = values % self.VALUE_MAX
        self._values = values

    @property
    def parameters(self):
        return self.values

    @parameters.setter
    def parameters(self, values):
        self.values = values

    @property
    def gates(self):
        return [gate for gate in self._gates]

    @property
    def parameter_instances(self):
        return self._params

    @property
    def num_param(self):
        return len(self._params)
    
    def to_string(self):
        ret = self._name + "|"
        ret += ",".join([g.gate.name for g in self.gates])
        ret += "/"
        ret += ",".join([str(g.qubit) for g in self.gates])
        ret += "/"
        ret += ",".join([f"{p:5.3f}" for p in self.parameters])
        return ret


In [45]:
# circuit
nq = 2
ng = 3
num_trial_unit = 9

# dataset
nx = 1
ny = 1
num_train_data = 20
num_test_data = 20
gateset = get_gateset(nq)

# optimization
shots = 50
num_iter =  100
variance = 0.3


In [46]:
unit = Unit.generate_random_unit("unit", nq, ng, gateset)


In [47]:
unit.to_string()


'unit|cz,ry,ry/1,0,1/0.000,0.000'

In [25]:
genome = ",".join([g.gate.name for g in unit.gates])
genome += "/" + ",".join(str(g.qubit) for g in unit.gates)
genome_val = ",".join([f"{p:5.3f}" for p in unit.parameters])
genome + "/" + genome_val


'cz,ry,ry/1,0,0/0.000,0.000'

In [48]:
strunit = unit.to_string()
print(strunit)


unit|cz,ry,ry/1,0,1/0.000,0.000


In [51]:
uname, strunit_vals = strunit.split("|")


In [53]:
gnames, gqubits, params = [
    strunit_part.split(",")
    for strunit_part in strunit_vals.split("/")
]
uname, gnames, gqubits, params


('unit', ['cz', 'ry', 'ry'], ['1', '0', '1'], ['0.000', '0.000'])